In [1]:
#@title
# Copyright 2020 Google LLC.

# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at

# https://www.apache.org/licenses/LICENSE-2.0

# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

In [1]:
#!pip install -q -U trax
import sys

# For example, if trax is inside a 'src' directory
project_root = '/raid/mmironczuk/projects/trax-upgrade'
sys.path.insert(0, project_root)

# Option to verify the import path
print(f"Python will look for packages in: {sys.path[0]}")

# Import trax
import trax

# Verify the source of the imported package
print(f"Imported trax from: {trax.__file__}")

Python will look for packages in: /raid/mmironczuk/projects/trax-upgrade
Imported trax from: /raid/mmironczuk/projects/trax-upgrade/trax/__init__.py


In [2]:
from trax import fastmath

# Use the tensorflow-numpy backend.
fastmath.set_backend(fastmath.Backend.JAX.value)
print(trax.fastmath.backend_name())
print(fastmath.jax.jax.devices())

2025-04-09 15:39:22.117015: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-04-09 15:39:22.145157: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-04-09 15:39:22.153895: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-04-09 15:39:22.173411: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-04-09 15:39:23.617740: W tensorflow/compiler/tf2

jax


In [3]:
# https://www.tensorflow.org/datasets/catalog/fashion_mnist
from trax.data.preprocessing import inputs as preprocessing
from trax.data.loader.tf import base as dataset

train_stream = dataset.TFDS('fashion_mnist', keys=('image', 'label'), train=True)()
eval_stream = dataset.TFDS('fashion_mnist', keys=('image', 'label'), train=False)()

2025-04-09 15:40:40.139994: W external/local_tsl/tsl/platform/cloud/google_auth_provider.cc:184] All attempts to get a Google authentication bearer token failed, returning an empty token. Retrieving token from files failed with "NOT_FOUND: Could not locate the credentials file.". Retrieving token from GCE failed with "FAILED_PRECONDITION: Error executing an HTTP request: libcurl code 6 meaning 'Couldn't resolve host name', error details: Could not resolve host: metadata.google.internal".


Dl Completed...: 0 url [00:00, ? url/s]
Dl Size...: 0 MiB [00:00, ? MiB/s]

Dl Completed...:   0%|          | 0/1 [00:00<?, ? url/s]
Dl Size...: 0 MiB [00:00, ? MiB/s]

Dl Completed...:   0%|          | 0/2 [00:00<?, ? url/s]
Dl Size...: 0 MiB [00:00, ? MiB/s]

Dl Completed...:   0%|          | 0/3 [00:00<?, ? url/s]
Dl Size...: 0 MiB [00:00, ? MiB/s]

Dl Completed...:   0%|          | 0/4 [00:00<?, ? url/s]
Dl Size...: 0 MiB [00:00, ? MiB/s]

Dl Completed...:   0%|          | 0/4 [00:00<?, ? url/s]
Dl Size...: 0 MiB [00:00, ? MiB/s]

Dl Completed...:   0%|          | 0/4 [00:00<?, ? url/s]
Dl Size...: 0 MiB [00:00, ? MiB/s]

Dl Size...:   0%|          | 0/4 [00:00<?, ? MiB/s]

Dl Size...:   0%|          | 0/4 [00:00<?, ? MiB/s]

Dl Size...:   0%|          | 0/29 [00:00<?, ? MiB/s]

Dl Size...:   0%|          | 0/29 [00:00<?, ? MiB/s]

Dl Size...:   0%|          | 0/29 [00:00<?, ? MiB/s]

Dl Size...:   0%|          | 0/29 [00:00<?, ? MiB/s]

Extraction completed...:   0%|          | 0/

Dataset fashion_mnist downloaded and prepared to /root/tensorflow_datasets/fashion_mnist/3.0.1. Subsequent calls will reuse this data.


In [4]:
train_data_pipeline = preprocessing.Serial(
    preprocessing.Shuffle(),
    preprocessing.Batch(8),
)

train_batches_stream = train_data_pipeline(train_stream)

eval_data_pipeline = preprocessing.Batch(8)
eval_batches_stream = eval_data_pipeline(eval_stream)

In [5]:
example_batch = next(train_batches_stream)
print(f'batch shape (image, label) = {[x.shape for x in example_batch]}')

batch shape (image, label) = [(8, 28, 28, 1), (8,)]


In [6]:
from trax import layers as tl


def get_model(n_output_classes=10):
    model = tl.Serial(
        tl.ToFloat(),

        tl.Conv(32, (3, 3), (1, 1), 'SAME'),
        tl.LayerNorm(),
        tl.Relu(),
        tl.MaxPool(),

        tl.Conv(64, (3, 3), (1, 1), 'SAME'),
        tl.LayerNorm(),
        tl.Relu(),
        tl.MaxPool(),

        tl.Flatten(),
        tl.Dense(n_output_classes),
    )
    return model

In [7]:
from trax.learning.supervised import training
from trax import optimizers as optimizers

train_task = training.TrainTask(
    labeled_data=train_batches_stream,
    loss_layer=tl.CategoryCrossEntropy(),
    optimizer=optimizers.Adam(0.01),
    n_steps_per_checkpoint=100,
)

eval_task = training.EvalTask(
    labeled_data=eval_batches_stream,
    metrics=[tl.CategoryCrossEntropy(), tl.CategoryAccuracy()],
    n_eval_batches=20,
)

In [19]:
model = get_model()

training_loop = training.Loop(model,
                              train_task,
                              eval_tasks=[eval_task],
                              output_dir='./cnn_model')

training_loop.run(1000)


Step      1: Total number of trainable weights: 451658
Step      1: Ran 1 train steps in 1.08 secs
Step      1: train CategoryCrossEntropy |  2.88087583
Step      1: eval  CategoryCrossEntropy |  178.71961899
Step      1: eval      CategoryAccuracy |  0.17500000

Step    100: Ran 99 train steps in 4.02 secs
Step    100: train CategoryCrossEntropy |  28.34364128
Step    100: eval  CategoryCrossEntropy |  6.03327532
Step    100: eval      CategoryAccuracy |  0.58125000

Step    200: Ran 100 train steps in 3.41 secs
Step    200: train CategoryCrossEntropy |  1.72970641
Step    200: eval  CategoryCrossEntropy |  0.85664938
Step    200: eval      CategoryAccuracy |  0.68125000

Step    300: Ran 100 train steps in 3.42 secs
Step    300: train CategoryCrossEntropy |  0.65041381
Step    300: eval  CategoryCrossEntropy |  0.82384256
Step    300: eval      CategoryAccuracy |  0.75000000

Step    400: Ran 100 train steps in 3.40 secs
Step    400: train CategoryCrossEntropy |  0.56752920
Step    

In [18]:
import shutil

shutil.rmtree(training_loop.output_dir)